In [2]:
import json
import pandas as pd
import numpy as np

In [3]:
spacy_df = pd.read_excel("spacy_df.xlsx")
reg_rec = pd.read_excel("Desktop/reg_rec.xlsx")
ner_labeling = pd.read_excel("Desktop/ner_labeling.xlsx")

In [4]:
spacy_df = spacy_df.drop("Unnamed: 0", axis=1)
reg_rec = reg_rec.drop("Unnamed: 0", axis=1)
ner_labeling = ner_labeling.drop("Unnamed: 0", axis=1)

In [ ]:
# clean_values = []
# for i in ner_labeling["Label"]:
#     i = re.sub(r'[\W_]', '', i)
#     clean_values.append(i)

# ner_labeling["Label"] = clean_values

In [7]:
ner_labeling_ORG = ner_labeling[ner_labeling.Label == "ORG"]
ner_labeling_PER = ner_labeling[ner_labeling.Label == "PER"]
ner_labeling_LOC = ner_labeling[ner_labeling.Label == "LOC"]

In [8]:
ner_labeling_clean = pd.concat([ner_labeling_ORG, ner_labeling_PER, ner_labeling_LOC], ignore_index = True)

## SpaCy

In [10]:
# merge the dataframes on the 'filenames' column
merged_df = pd.merge(spacy_df, ner_labeling_clean, how = "inner")

# count the number of rows in the merged dataframe (which will be the number of identical rows)
num_identical_rows = len(merged_df)

print(f'The two dataframes have {num_identical_rows} identical rows.')

The two dataframes have 5124 identical rows.


In [11]:
# same entities, different tags
diff_merged_df = pd.merge(spacy_df, ner_labeling_clean, on=['Filename','Text'])

# filter the merged dataframe to include only rows where the 'labels' values are different
diff_labels_df = diff_merged_df[diff_merged_df['Label_x'] != diff_merged_df['Label_y']]

# count the number of rows in the filtered dataframe (which will be the number of rows with the same 'text' value but different 'labels' values)
num_diff_rows = len(diff_labels_df)

print(f'The two dataframes have {num_diff_rows} rows with the same "text" value but different "labels" values.')

The two dataframes have 197 rows with the same "text" value but different "labels" values.


In [26]:
# Metrics compared to only 5124 entities of the same boundaries (could be different tags)
import sklearn
from sklearn.metrics import classification_report

print(classification_report(diff_merged_df["Label_y"], diff_merged_df["Label_x"]))

              precision    recall  f1-score   support

         LOC       0.82      0.91      0.87       380
         ORG       0.96      1.00      0.98      2719
         PER       1.00      0.93      0.96      2222

    accuracy                           0.96      5321
   macro avg       0.93      0.95      0.94      5321
weighted avg       0.96      0.96      0.96      5321



Strict predictions 

In [29]:
# Доля 5124 (правильно найденных спейси) / 35706 (все что нашел спейси)
precision = 0.14350529

# Доля 5124 (правильных найденных спейси)/ 17925 (все которые нужно было найти)
recall = 0.28585774

In [30]:
F1 = 2 * (precision * recall) / (precision + recall)
F1

0.19108351213864222

In [41]:
precision_org = len(merged_df[merged_df.Label == "ORG"]) / len(spacy_df[spacy_df.Label == "ORG"])
recall_org = len(merged_df[merged_df.Label == "ORG"]) / len(ner_labeling_clean[ner_labeling_clean.Label == "ORG"])
f1_org = 2 * (precision_org * recall_org) / (precision_org + recall_org)

In [57]:
len(ner_labeling_clean[ner_labeling_clean.Label == "ORG"])

6403

In [50]:
print(precision_org, recall_org, f1_org)

0.18222222222222223 0.4226143995002343 0.2546464028607726


In [51]:
precision_loc = len(merged_df[merged_df.Label == "LOC"]) / len(spacy_df[spacy_df.Label == "LOC"])
recall_loc = len(merged_df[merged_df.Label == "LOC"]) / len(ner_labeling_clean[ner_labeling_clean.Label == "LOC"])
f1_loc = 2 * (precision_loc * recall_loc) / (precision_loc + recall_loc)

In [52]:
print(precision_loc, recall_loc, f1_loc)

0.02722422720853601 0.06156848828956707 0.03775432488303775


In [53]:
len(ner_labeling_clean[ner_labeling_clean.Label == "LOC"])

5636

In [54]:
precision_per = len(merged_df[merged_df.Label == "PER"]) / len(spacy_df[spacy_df.Label == "PER"])
recall_per = len(merged_df[merged_df.Label == "PER"]) / len(ner_labeling_clean[ner_labeling_clean.Label == "PER"])
f1_per = 2 * (precision_per * recall_per) / (precision_per + recall_per)

In [55]:
print(precision_per, recall_per, f1_per)

0.255363748458693 0.35185185185185186 0.2959416976278937


In [56]:
len(ner_labeling_clean[ner_labeling_clean.Label == "PER"])

5886

In [ ]:
from fuzzywuzzy import fuzz
def match_entities(df_ann, df_dp):
    matches = []
    for _, row_ann in df_ann.iterrows():
        for _, row_dp in df_dp.iterrows():
            if row_ann['Filename'] == row_dp['Filename'] and row_ann['Label'] == row_dp['Label'] and fuzz.ratio(row_ann['Text'], row_dp['Text']) >= 80:
                matches.append((row_ann.name, row_dp.name))
                print(len(matches))
    return matches

In [ ]:
spacy_matches = match_entities(ner_labeling_clean, spacy_df)

In [186]:
tp = len(spacy_matches)
fp = len(dp_df) - tp
fn = len(ner_labeling_clean) - tp
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * precision * recall / (precision + recall)

In [187]:
# Define the classes to calculate metrics for
classes = ["ORG", "LOC", "PER"]

# Calculate true positives, false positives, and false negatives for each class
tp_by_class = {c: 0 for c in classes}
fp_by_class = {c: 0 for c in classes}
fn_by_class = {c: 0 for c in classes}
for i, row in ner_labeling_clean.iterrows():
    match_found = False
    for match in spacy_matches:
        if match[0] == i:
            match_found = True
            j = match[1]
            if row["Label"] == spacy_df.iloc[j]["Label"]:
                tp_by_class[row["Label"]] += 1
            else:
                fn_by_class[row["Label"]] += 1
    if not match_found:
        fn_by_class[row["Label"]] += 1

for j, row in spacy_df.iterrows():
    match_found = False
    for match in spacy_matches:
        if match[1] == j:
            match_found = True
    if not match_found:
        fp_by_class[row["Label"]] += 1

# Calculate precision, recall, and F1 score for each class
for c in classes:
    precision = tp_by_class[c] / (tp_by_class[c] + fp_by_class[c])
    recall = tp_by_class[c] / (tp_by_class[c] + fn_by_class[c])
    f1_score = 2 * precision * recall / (precision + recall)

    print("Class:", c)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 score:", f1_score)

# Calculate overall precision, recall, and F1 score
tp = sum(tp_by_class.values())
fp = sum(fp_by_class.values())
fn = sum(fn_by_class.values())
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * precision * recall / (precision + recall)

print("Overall precision:", precision)
print("Overall recall:", recall)
print("Overall F1 score:", f1_score)

Class: ORG
Precision: 0.492492624719299
Recall: 0.8176169590643275
F1 score: 0.6147124288972549
Class: LOC
Precision: 0.19614243323442138
Recall: 0.3581685180167976
F1 score: 0.25347521809989454
Class: PER
Precision: 0.5477722979185823
Recall: 0.7360440301022128
F1 score: 0.6281031342854404
Overall precision: 0.4232670183162354
Overall recall: 0.6801935591523444
Overall F1 score: 0.5218192757203753


## Natasha

In [168]:
natasha = json.load(open('Desktop/natasha/natasha.json', 'rb'))

In [169]:
rows = []

# loop through the dictionary
for filename, (text, tags) in natasha.items():
    # create a row with the filename, text, and tags
    for i in range(len(text)):
        row = [filename, text[i], tags[i]]
        # add the row to the list of rows
        rows.append(row)

# create a dataframe from the list of rows
natasha_df = pd.DataFrame(rows, columns=['Filename', 'Text', 'Label'])

In [67]:
# merge the dataframes on the 'filenames' column
nat_df_merged = pd.merge(natasha_df, ner_labeling_clean, how = "inner")

# count the number of rows in the merged dataframe (which will be the number of identical rows)
num_identical_rows = len(nat_df_merged)

print(f'The two dataframes have {num_identical_rows} identical rows.')

The two dataframes have 7312 identical rows.


In [68]:
# same entities, different tags
nat_diff_merged_df = pd.merge(natasha_df, ner_labeling_clean, on=['Filename','Text'])

# filter the merged dataframe to include only rows where the 'labels' values are different
nat_diff_labels_df = nat_diff_merged_df[nat_diff_merged_df['Label_x'] != nat_diff_merged_df['Label_y']]
# count the number of rows in the filtered dataframe (which will be the number of rows with the same 'text' value but different 'labels' values)
num_diff_rows = len(nat_diff_labels_df)

print(f'The two dataframes have {num_diff_rows} rows with the same "text" value but different "labels" values.')

The two dataframes have 88 rows with the same "text" value but different "labels" values.


In [69]:
# Lenient F1
import sklearn
from sklearn.metrics import classification_report

print(classification_report(nat_diff_merged_df["Label_y"], nat_diff_merged_df["Label_x"]))

              precision    recall  f1-score   support

         LOC       0.98      0.93      0.95       390
         ORG       0.97      1.00      0.98      2368
         PER       1.00      0.99      0.99      4642

    accuracy                           0.99      7400
   macro avg       0.98      0.97      0.98      7400
weighted avg       0.99      0.99      0.99      7400



In [73]:
precision_org = len(nat_df_merged[nat_df_merged.Label == "ORG"]) / len(natasha_df[natasha_df.Label == "ORG"])
recall_org = len(nat_df_merged[nat_df_merged.Label == "ORG"]) / len(ner_labeling_clean[ner_labeling_clean.Label == "ORG"])
f1_org = 2 * (precision_org * recall_org) / (precision_org + recall_org)
print(precision_org, recall_org, f1_org)

0.1651305683563748 0.3693581133843511 0.22822677925211093


In [74]:
len(ner_labeling_clean[ner_labeling_clean.Label == "ORG"])

6403

In [75]:
precision_loc = len(nat_df_merged[nat_df_merged.Label == "LOC"]) / len(natasha_df[natasha_df.Label == "LOC"])
recall_loc = len(nat_df_merged[nat_df_merged.Label == "LOC"]) / len(ner_labeling_clean[ner_labeling_clean.Label == "LOC"])
f1_loc = 2 * (precision_loc * recall_loc) / (precision_loc + recall_loc)
print(precision_loc, recall_loc, f1_loc)
len(ner_labeling_clean[ner_labeling_clean.Label == "LOC"])

0.02982499383781119 0.06440738112136267 0.040770483517717754


5636

In [76]:
precision_per = len(nat_df_merged[nat_df_merged.Label == "PER"]) / len(natasha_df[natasha_df.Label == "PER"])
recall_per = len(nat_df_merged[nat_df_merged.Label == "PER"]) / len(ner_labeling_clean[ner_labeling_clean.Label == "PER"])
f1_per = 2 * (precision_per * recall_per) / (precision_per + recall_per)
print(precision_per, recall_per, f1_per)
len(ner_labeling_clean[ner_labeling_clean.Label == "PER"])

0.6942298955020445 0.7787971457696228 0.7340859956761949


5886

In [ ]:
natasha_matches = match_entities(ner_labeling_clean, natasha_df)

tp = len(natasha_matches)
fp = len(dp_df) - tp
fn = len(ner_labeling_clean) - tp
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * precision * recall / (precision + recall)

In [184]:
# Define the classes to calculate metrics for
classes = ["ORG", "LOC", "PER"]

# Calculate true positives, false positives, and false negatives for each class
tp_by_class = {c: 0 for c in classes}
fp_by_class = {c: 0 for c in classes}
fn_by_class = {c: 0 for c in classes}
for i, row in ner_labeling_clean.iterrows():
    match_found = False
    for match in natasha_matches:
        if match[0] == i:
            match_found = True
            j = match[1]
            if row["Label"] == natasha_df.iloc[j]["Label"]:
                tp_by_class[row["Label"]] += 1
            else:
                fn_by_class[row["Label"]] += 1
    if not match_found:
        fn_by_class[row["Label"]] += 1

for j, row in natasha_df.iterrows():
    match_found = False
    for match in natasha_matches:
        if match[1] == j:
            match_found = True
    if not match_found:
        fp_by_class[row["Label"]] += 1

# Calculate precision, recall, and F1 score for each class
for c in classes:
    precision = tp_by_class[c] / (tp_by_class[c] + fp_by_class[c])
    recall = tp_by_class[c] / (tp_by_class[c] + fn_by_class[c])
    f1_score = 2 * precision * recall / (precision + recall)

    print("Class:", c)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 score:", f1_score)

# Calculate overall precision, recall, and F1 score
tp = sum(tp_by_class.values())
fp = sum(fp_by_class.values())
fn = sum(fn_by_class.values())
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * precision * recall / (precision + recall)

print("Overall precision:", precision)
print("Overall recall:", recall)
print("Overall F1 score:", f1_score)

Class: ORG
Precision: 0.4790281808819399
Recall: 0.8275109170305677
F1 score: 0.606795540796964
Class: LOC
Precision: 0.2083236815689908
Recall: 0.366227772498982
F1 score: 0.2655773206024215
Class: PER
Precision: 0.7376906662451592
Recall: 0.863619541080681
F1 score: 0.7957035079493627
Overall precision: 0.47406634586722307
Overall recall: 0.7290200058937166
Overall F1 score: 0.5745287522417329


## DeepPavlov

In [4]:
import json
import pandas as pd
dp = json.load(open('Desktop/deep_pavlov.json', 'rb'))

In [5]:
rows = []

# loop through the dictionary
for filename, (text, tags) in dp.items():
    # create a row with the filename, text, and tags
    for i in range(len(text)):
        row = [filename, text[i], tags[i]]
        # add the row to the list of rows
        rows.append(row)

# create a dataframe from the list of rows
dp_df = pd.DataFrame(rows, columns=['Filename', 'Text', 'Label'])

In [6]:
dp_df = dp_df.explode(['Text', 'Label'])

In [7]:
dp_df = dp_df[dp_df.Label != "O"]

In [10]:
import openpyxl
dp_df.to_excel('dp_df_full.xlsx')

In [163]:
# Concat entities
dp_df['Entity'] = dp_df['Label'].apply(lambda x: x.split('-')[1] if '-' in x else '')

# Group by filename and entity, and concatenate the 'text' and 'label' columns
dp_df = dp_df.groupby(['Filename', 'Entity']).agg({'Text': ' '.join, 'Label': 'first'}).reset_index()

In [120]:
# merge the dataframes on the 'filenames' column
dp_df_merged = pd.merge(dp_df, ner_labeling_clean, how = "inner")

# count the number of rows in the merged dataframe (which will be the number of identical rows)
num_identical_rows = len(dp_df_merged)

print(f'The two dataframes have {num_identical_rows} identical rows.')

The two dataframes have 0 identical rows.


In [ ]:
def match_entities(df_ann, df_dp):
    matches = []
    for _, row_ann in df_ann.iterrows():
        for _, row_dp in df_dp.iterrows():
            if row_ann['Label'] == row_dp['Entity'] and fuzz.ratio(row_ann['Text'], row_dp['Text']) >= 80:
                matches.append((row_ann.name, row_dp.name))
                print(len(matches))
    return matches

matches = match_entities(ner_labeling_clean, dp_df)

tp = len(matches)
fp = len(dp_df) - tp
fn = len(ner_labeling_clean) - tp
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * precision * recall / (precision + recall)

In [142]:
ground_truth_df = ner_labeling_clean
recognized_df = dp_df

In [146]:
# Define the classes to calculate metrics for
classes = ["ORG", "LOC", "PER"]

# Calculate true positives, false positives, and false negatives for each class
tp_by_class = {c: 0 for c in classes}
fp_by_class = {c: 0 for c in classes}
fn_by_class = {c: 0 for c in classes}
for i, row in ground_truth_df.iterrows():
    match_found = False
    for match in matches:
        if match[0] == i:
            match_found = True
            j = match[1]
            if row["Label"] == recognized_df.iloc[j]["Entity"]:
                tp_by_class[row["Label"]] += 1
            else:
                fn_by_class[row["Label"]] += 1
    if not match_found:
        fn_by_class[row["Label"]] += 1

for j, row in recognized_df.iterrows():
    match_found = False
    for match in matches:
        if match[1] == j:
            match_found = True
    if not match_found:
        fp_by_class[row["Entity"]] += 1

# Calculate precision, recall, and F1 score for each class
for c in classes:
    precision = tp_by_class[c] / (tp_by_class[c] + fp_by_class[c])
    recall = tp_by_class[c] / (tp_by_class[c] + fn_by_class[c])
    f1_score = 2 * precision * recall / (precision + recall)

    print("Class:", c)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 score:", f1_score)

# Calculate overall precision, recall, and F1 score
tp = sum(tp_by_class.values())
fp = sum(fp_by_class.values())
fn = sum(fn_by_class.values())
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * precision * recall / (precision + recall)

print("Overall precision:", precision)
print("Overall recall:", recall)
print("Overall F1 score:", f1_score)

Class: ORG
Precision: 0.8385991058122205
Recall: 0.5516666666666666
F1 score: 0.6655233589591957
Class: LOC
Precision: 0.4
Recall: 0.11938775510204082
F1 score: 0.18388998035363457
Class: PER
Precision: 0.11858704793944491
Recall: 0.023809523809523808
F1 score: 0.0396568696385881
Overall precision: 0.6701885228920654
Overall recall: 0.2940641759840015
Overall F1 score: 0.4087692696487238


In [166]:
dp_df['Label'].unique()

array(['B-LOC', 'B-ORG', 'B-PER', 'S-LOC', 'S-PER', 'S-ORG', 'E-ORG',
       'I-ORG', 'I-PER', 'E-PER'], dtype=object)

In [180]:
import openpyxl
natasha_df.to_excel('natasha_df.xlsx')
spacy_df.to_excel('spacy_df.xlsx')
dp_df.to_excel('dp_df.xlsx')
ner_labeling_clean.to_excel('ner_labeling_clean.xlsx')